In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
ROOT_PATH='/content/drive/My Drive/Colab Notebooks/learning_ai'
sys.path.append(ROOT_PATH)

Mounted at /content/drive


In [1]:
import sys
ROOT_PATH='.'
sys.path.append(ROOT_PATH)

In [15]:
!pip install dlt

###VGG16+GAP+batchnorm

In [2]:
from common import config
config.GPU=True
from common.np import *
from model.base_model import BaseModel
from common.optimizer import *
from common.trainer import Trainer
import dlt
from common.util import to_cpu,to_gpu

data = dlt.cifar.load_cifar10()
x_train=data.train_images
x_test=data.test_images
t_train=data.train_labels
t_test=data.test_labels

x_train=x_train.transpose(0,3,1,2).astype('f')
x_test=x_test.transpose(0,3,1,2).astype('f')
t_train=t_train.reshape(t_train.shape[0])
t_test=t_test.reshape(t_test.shape[0])

if GPU:
    x_train=to_gpu(x_train)
    x_test=to_gpu(x_test)
    t_train=to_gpu(t_train)
    t_test=to_gpu(t_test)

x_train/=255.
x_test/=255.

input_shape=(3,32,32)

layer_list=[['conv',[64,3,3,1,1,2]], ['batchnorm',[0.9,None,None]],['relu',[]],
            ['conv',[64,3,3,1,1,2]],['batchnorm',[0.9,None,None]],['relu',[]],['pool',[2,2,0,2]],
            ['conv',[128,3,3,1,1,2]],['batchnorm',[0.9,None,None]], ['relu',[]],
            ['conv',[128,3,3,1,1,2]],['batchnorm',[0.9,None,None]],['relu',[]],['pool',[2,2,0,2]],
            ['conv',[256,3,3,1,1,2]],['batchnorm',[0.9,None,None]], ['relu',[]],
            ['conv',[256,3,3,1,1,2]],['batchnorm',[0.9,None,None]],['relu',[]],
            ['conv',[256,3,3,1,1,2]],['batchnorm',[0.9,None,None]],['relu',[]],['pool',[2,2,0,2]],
            ['conv',[512,3,3,1,1,2]],['batchnorm',[0.9,None,None]], ['relu',[]],
            ['conv',[512,3,3,1,1,2]],['batchnorm',[0.9,None,None]],['relu',[]],
            ['conv',[512,3,3,1,1,2]],['batchnorm',[0.9,None,None]],['relu',[]],['pool',[2,2,0,2]],
            ['conv',[512,3,3,1,1,2]], ['batchnorm',[0.9,None,None]],['relu',[]],
            ['conv',[512,3,3,1,1,2]],['batchnorm',[0.9,None,None]],['relu',[]],
            ['conv',[512,3,3,1,1,2]],['batchnorm',[0.9,None,None]],['relu',[]],
            ['gap',[]],['affine',[10,2]],['relu',[]]]

lr=0.001
max_epoch=20
batch_size=64
clip_grads=None
eval_interval=100
eval_accuracy=True
eval_sample_num=1000
weight_decay='ridge'
weight_decay_lambda=1e-4
show_distribution=False
file_name=ROOT_PATH+'/pkl/weight_decay_ridge_cifar10/lambda_1e-4/weight_decay_ridge_cifar10'

args_model=(input_shape,layer_list,weight_decay,weight_decay_lambda,show_distribution)
args_trainer=(x_train,t_train,max_epoch,batch_size,clip_grads,eval_interval,eval_accuracy,eval_sample_num,x_test,t_test)

model=BaseModel(*args_model)
for layer in model.layers:
    print(layer.__class__.__name__,layer.output_shape)
optimizer=Adam(lr)
trainer=Trainer(model,optimizer,file_name)

trainer.fit(*args_trainer)

------------------------------------------------------------
                       GPU Mode (cupy)
------------------------------------------------------------

Convolution (64, 32, 32)
BatchNormalization (64, 32, 32)
Relu (64, 32, 32)
Convolution (64, 32, 32)
BatchNormalization (64, 32, 32)
Relu (64, 32, 32)
Pooling (64, 16, 16)
Convolution (128, 16, 16)
BatchNormalization (128, 16, 16)
Relu (128, 16, 16)
Convolution (128, 16, 16)
BatchNormalization (128, 16, 16)
Relu (128, 16, 16)
Pooling (128, 8, 8)
Convolution (256, 8, 8)
BatchNormalization (256, 8, 8)
Relu (256, 8, 8)
Convolution (256, 8, 8)
BatchNormalization (256, 8, 8)
Relu (256, 8, 8)
Convolution (256, 8, 8)
BatchNormalization (256, 8, 8)
Relu (256, 8, 8)
Pooling (256, 4, 4)
Convolution (512, 4, 4)
BatchNormalization (512, 4, 4)
Relu (512, 4, 4)
Convolution (512, 4, 4)
BatchNormalization (512, 4, 4)
Relu (512, 4, 4)
Convolution (512, 4, 4)
BatchNormalization (512, 4, 4)
Relu (512, 4, 4)
Pooling (512, 2, 2)
Convolution (512, 2